In [3]:
import altair as alt
import pandas as pd
import random

In [56]:
# definim el primer valor de la llista
current_value = random.uniform(-0.8, 1)

# definim la llista buida
values = []

# iterem 1000 vegades per generar els valors de la llista
for i in range(300):
    # afegim el valor actual a la llista
    values.append(current_value)
    
    # calculem el valor següent amb un pas màxim de 0.05
    next_value = min(max(current_value + random.uniform(-0.04, 0.04), -1.2), 1)
    
    # actualitzem el valor actual per al següent pas del bucle
    current_value = next_value


In [57]:
# llegim el fitxer CSV amb pandas
data = pd.DataFrame([
    {'temps':i, 'distancia':values[i], 'sector': i//100} for i in range(300)]
)

display(data.head())

# definim la visualització de dades
chart = alt.Chart(data).mark_bar().encode(
    # alt x with a ratio from -1 to 1
    x=alt.X('distancia:Q', scale=alt.Scale(domain=[-1, 1])),
    y='temps:T',
    color=alt.condition(
        alt.datum.distancia > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    ),
    order='temps:Q',
    tooltip=['sector:N', 'distancia:Q']
)

sectors = pd.DataFrame([{'y': 0, 'label': 'Sector 1'}, {'y': 100, 'label': 'Sector 2'}, {'y': 200, 'label': 'Sector 3'}])
rule = alt.Chart(sectors).mark_rule(color='red').encode(
    y=alt.Y('y:T', axis=alt.Axis(title='temps')),
)
text = rule.mark_text(
    align='right',
    baseline='middle',
    dx=+200,
    dy=-5
).encode(
    text='label:N'
)
chart + rule + text


,temps,distancia,sector
0,0,0.314729,0
1,1,0.289216,0
2,2,0.268415,0
3,3,0.305455,0
4,4,0.294346,0


/usr/local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)